In [61]:
from sklearn.tree import DecisionTreeRegressor
import nilmtk
from nilmtk import DataSet, TimeFrame
import math


train = DataSet('/nilmtk/data/REDD/redd.h5') 
test = DataSet('/nilmtk/data/REDD/redd.h5') 
building_number = 1

mains = train.buildings[building_number].elec.mains()
df_all = mains.power_series_all_data() #df_all has a bunch of NaNs
df_all_noNan = df_all.dropna()
a = df_all_noNan.keys()
middleTime = a[int(math.floor(a.size/2))]
middleTimeStr = "%d-%02d-%02d %02d:%02d:%02d" % (middleTime.year, middleTime.month, middleTime.day, middleTime.hour, middleTime.minute, middleTime.second)

train.set_window(end=middleTimeStr)
test.set_window(start=middleTimeStr)

train_elec = train.buildings[building_number].elec
test_elec = test.buildings[building_number].elec

top_train_elec = train_elec.submeters().select_top_k(k=5)

Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
15/16 MeterGroup(meters=
  ElecMeter(instance=3, building=1, dataset='REDD', appliances=[Appliance(type='electric oven', instance=1)])
  ElecMeter(instance=4, building=1, dataset='REDD', appliances=[Appliance(type='electric oven', instance=1)])
16/16 MeterGroup(meters=
  ElecMeter(instance=10, building=1, dataset='REDD', appliances=[Appliance(type='washer dryer', instance=1)])
  ElecMeter(instance=20, building=1, dataset='REDD', appliances=[Appliance(type='washer dryer', instance=1)])
Calculating total_energy for ElecMeterID(instance=20, building=1, dataset='REDD') ...   

In [62]:
from nilmtk.disaggregate import CombinatorialOptimisation
co = CombinatorialOptimisation() #mk change this later  to default
co.train(top_train_elec, sample_period=60, resample=True)

Training model for submeter 'ElecMeter(instance=5, building=1, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])'
Training model for submeter 'ElecMeter(instance=11, building=1, dataset='REDD', appliances=[Appliance(type='microwave', instance=1)])'
Training model for submeter 'ElecMeter(instance=8, building=1, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])'
Training model for submeter 'ElecMeter(instance=9, building=1, dataset='REDD', appliances=[Appliance(type='light', instance=1)])'
Training model for submeter 'ElecMeter(instance=6, building=1, dataset='REDD', appliances=[Appliance(type='dish washer', instance=1)])'
Done training!


In [65]:
from nilmtk.disaggregate import DecisionTree
import numpy as np
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTree()
#dt.train(top_train_elec)
metergroup = top_train_elec
min_samples_split=np.arange(2, 400, 10)
pred = {}

mains_power_series = train_elec.mains().power_series(sample_period=60, resample=True)
mains_chunk = next(mains_power_series)

for i, meter in enumerate(metergroup.submeters().meters):
    #print("Training model for submeter '{}'".format(meter))
    power_series = meter.power_series(sample_period=60, resample=True) #**load_kwargs
    chunk = next(power_series)
    list_clfs = []
    losses = []
    x_train = mains_chunk.values
    y_train = chunk.values
    print("{} -- {}".format(x_train.shape,y_train.shape))
    
"""    for split in min_samples_split:
        clf = DecisionTreeRegressor(min_samples_split = split)
        clf.fit(x_train, y_train)"""

Loading data for meter ElecMeterID(instance=2, building=1, dataset='REDD')     
Done loading data all meters for this chunk.


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').